In [14]:
import sys
print(sys.version)

3.6.0 |Anaconda 4.3.1 (64-bit)| (default, Dec 23 2016, 11:57:41) [MSC v.1900 64 bit (AMD64)]


In [1]:
import pandas as pd
import numpy as np

In [2]:
from gensim.models import Word2Vec

C:\Users\barte\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
from sklearn.manifold import t_sne

#### Data loading

In [4]:
data = pd.read_csv("titles_books.csv")
data.sample(10)

,Unnamed: 0,title
37663,6844474730,Pan Poniedziałek Klucze do królestwa - NIX
38047,6844795218,ZAGINIONA PILIPIUK
10043,6507601770,Boski ogień Staveley + Krąg Elfgren fantasy magia
35064,6841603932,Czerwona Królowa + Szklany Miecz Victoria Aveyard
36579,6843607197,! KRWAWA HRABINA PAJĘCZA BOGINI Tara Moss
23392,6785909116,KLEJNOT W CZASZCE Michael Moorcock*
36200,6843074433,Taniec nad przepaścią Elena Malinowska
6265,6044450377,Księżniczka - Pilipiuk Andrzej
33982,6840301188,EUGENIUSZ DĘBSKI - O WŁOS OD PIWA
10655,6563991333,PIOTR ZWIERZCHOWSKI SENDILKELM _____!


In [5]:
import re
def getWords(text):
    all_words = re.compile('\w+').findall(text.replace(".", ""))
    return [words.lower() for words in all_words if len(words)>1]

In [6]:
titles = [getWords(title) for title in data.title]

#### Model training

In [7]:
VECTOR_DIM = 100

In [22]:
model = Word2Vec(titles, size=VECTOR_DIM, min_count=0)

In [23]:
model.similar_by_word(word="tolkien".lower(), topn=20)

[('hobbit', 0.9975394010543823),
 ('jrr', 0.9969319105148315),
 ('silmarillion', 0.9952049255371094),
 ('pierścienia', 0.9946894645690918),
 ('powrotem', 0.9936426877975464),
 ('władca', 0.9911157488822937),
 ('pierścieni', 0.9903944134712219),
 ('hurina', 0.9887254238128662),
 ('dwie', 0.9880675673484802),
 ('tam', 0.9869142770767212),
 ('kropce', 0.986514687538147),
 ('kropka', 0.9840320348739624),
 ('mórz', 0.9836744666099548),
 ('beowulf', 0.9832788705825806),
 ('wieze', 0.9830971360206604),
 ('jrrtolkien', 0.9822474718093872),
 ('tomie', 0.9818856716156006),
 ('dżil', 0.98114013671875),
 ('trygław', 0.9793127179145813),
 ('czyli', 0.9790645837783813)]

In [24]:
def get_word_score(model, word):
    try:
        score = model.wv[word]
    except:
        score = np.repeat(np.nan, repeats=VECTOR_DIM)
    return score

def get_title_score(model, title):
    title_parsed = getWords(title)
    mean_score = np.nanmean([get_word_score(model, i) for i in title_parsed], axis = 0) #nanmean to jak na.rm=TRUE w R
    return mean_score

In [25]:
v1 = get_title_score(model, "Tolkien - Władca pierścieni - Trylogia")
v2 = get_title_score(model, "NAUKA ŚWIATA DYSKU I - Terry Pratchett")
np.sum((v1-v2)**2)

80.005814

In [26]:
vectorized_titles = [get_title_score(model, title) for title in data.title]

#### T-SNE visualizations

In [27]:
tsne_transformer = t_sne.TSNE()

In [ ]:
tnsed_titles = tsne_transformer.fit_transform(vectorized_titles)